---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [180]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [88]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [89]:
#cleaning the university_towns.txt
file =open('university_towns.txt')
l = []
st = []
x=1
i=0
d=dict()
for line in file:
    word = line.strip()
    l.append(word)
while i < len(l):
    if '[edit]' in l[i]:
        d[l[i].replace('[edit]','')] = []
    i=i+1
for i in sorted(list(d.keys())):
    for j in range(x,len(l)):
        if '[edit]' in l[j]:
            x=j+1
            break
        else:
            d[i].append(l[j])
for i in d.keys():
    st.append(DataFrame(d[i],index=[i]*len(d[i])))
dat=pd.concat(st)
dat[1]=dat[0]
dat[0]=dat.index
dat.index=range(0,len(dat.index))
dat.columns=['State','RegionName']
def function(words):
    l=[]
    for i in words:
        if i =='(':
             break
        else:
            l.append(i)
    word= ''.join(l)
    if word[-1]==' ':
        word=word[:len(word)-1]
    else:
        word = word
    return word
dat['RegionName']=dat['RegionName'].map(function)
dat=dat.sort_index(by='State')
dat.index=range(0,517)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:43: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [90]:
def get_list_of_university_towns():
    return dat

In [120]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Montevallo
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Tucson


In [92]:
#cleaning the GDP data
dat1=pd.read_excel('gdplev.xls')
dat1=dat1[['Unnamed: 4','Unnamed: 6']]
dat1=dat1.ix[219:,:]
dat1.columns=['Quarterly','GDP']
dat1.index=range(0,len(dat1.index))
gdp=dat1
gdp.head()

,Quarterly,GDP
0,2000q1,12359.1
1,2000q2,12592.5
2,2000q3,12607.7
3,2000q4,12679.3
4,2001q1,12643.3


In [93]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    l=[]
    for i in range(len(gdp['GDP'].values)-2):
        if (gdp['GDP'].values[i]>gdp['GDP'].values[i+1]) and (gdp['GDP'].values[i+1]>gdp['GDP'].values[i+2]):
            l.append(gdp['Quarterly'][i])
    return l[1]

In [94]:
get_recession_start()

'2008q3'

In [97]:
def get_recession_end():
    l=[]
    for i in range(len(gdp['GDP'].values)-2):
        if (gdp['GDP'].values[i]>gdp['GDP'].values[i+1]) and (gdp['GDP'].values[i+1]>gdp['GDP'].values[i+2]) and (gdp['GDP'].values[i+2]<gdp['GDP'].values[i+3]):
            l.append(gdp['Quarterly'][i+4])
    return l[0]
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
  

In [98]:
get_recession_end()

'2009q4'

In [99]:
def get_recession_bottom():
    start=gdp.ix[gdp['Quarterly']==get_recession_start(),:].index[0]
    end=gdp.ix[gdp['Quarterly']==get_recession_end(),:].index[0]
    min_gdp=gdp.ix[start:end,:]['GDP'].min()
    return gdp.ix[gdp['GDP']==min_gdp,:]['Quarterly'].values[0]
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
   

In [100]:
get_recession_bottom()

'2009q2'

In [101]:
dat2=pd.read_csv('City_Zhvi_AllHomes.csv')
f = lambda x:states[x]
dat2['State']=dat2['State'].map(f)
dat2=dat2.sort_index(by='State')
dat2=dat2.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
dat2=dat2.ix[:,dat2.columns>='2000-01']
l=[]
x=list(gdp['Quarterly'].values)+['2016q3']
for i in range(len(x)):
    a=(x[i]+' ')*3
    a=a.split(' ')[:3]
    l +=a
l=l[:len(l)-1]
dat2.columns=['RegionName','State']+l
new=DataFrame()
for i in x:
    new[i]=dat2[i].mean(axis=1)
new['RegionName']=dat2['RegionName']
new['State']=dat2['State']
new=new.set_index(["State","RegionName"])

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [102]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return new

In [103]:
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   164400.000000  165900.000000  167600.000000   
        Satsuma       88600.000000   90966.666667   90333.333333   
        Irvington     72466.666667   74100.000000   75700.000000   
        Mount Olive  106500.000000  107333.333333  106966.666667   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   169766.666667  172533.333333  173433.333333   
        Satsuma       89600.000000   90233.333333   90766.666667   
        Irvington     76266.666667   76300.000000   74300.000000   
        Mount Olive  107200.000000  108266.666667  108766.666667   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   174166.666667  176700.000000  178000.000000   
        Satsuma       90633.333333   92200.000000   93000.000000   
        Irvington     73500.000000   73833.333333   75966.666667   
        Mount Olive  110166.666667  112400.000000  113966.666667   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Mulga                  NaN    ...      45466.666667   43600.000000   
        Trussville   180800.000000    ...     212033.333333  210966.666667   
        Satsuma       93200.000000    ...     125266.666667  127500.000000   
        Irvington     79266.666667    ...      90533.333333   95666.666667   
        Mount Olive  115000.000000    ...     132066.666667  132366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Mulga         42166.666667   41933.333333   42733.333333   
        Trussville   214566.666667  216600.000000  220066.666667   
        Satsuma      128266.666667  129266.666667  131300.000000   
        Irvington     98466.666667   99500.000000   98800.000000   
        Mount Olive  132066.666667  135433.333333  138100.000000   

                            2015q3         2015q4         2016q1  \
State   RegionName                                                 
Alabama Mulga         42033.333333   43033.333333   40666.666667   
        Trussville   219833.333333  219433.333333  220266.666667   
        Satsuma      133333.333333  134800.000000  134666.666667   
        Irvington     97600.000000   98000.000000  101500.000000   
        Mount Olive  138433.333333  140633.333333  142866.666667   

                            2016q2    2016q3  
State   RegionName                            
Alabama Mulga         38366.666667   38300.0  
        Trussville   223566.666667  225600.0  
        Satsuma      133533.333333  133100.0  
        Irvington    103966.666667  103950.0  
        Mount Olive  144033.333333  143850.0  

[5 rows x 67 columns]

In [104]:
dat3=pd.read_csv('City_Zhvi_AllHomes.csv')
f = lambda x:states[x]
dat3['State']=dat3['State'].map(f)
dat3=dat3.sort_index(by='State')
dat3=dat3.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
dat3=dat3.ix[:,dat3.columns>='2000-01']
l=[]
x=list(gdp['Quarterly'].values)+['2016q3']
for i in range(len(x)):
    a=(x[i]+' ')*3
    a=a.split(' ')[:3]
    l +=a
l=l[:len(l)-1]
dat3.columns=['RegionName','State']+l
new1=DataFrame()
for i in x:
    new1[i]=dat3[i].mean(axis=1)
new1['RegionName']=dat3['RegionName']
new1['State']=dat3['State']

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [178]:

new1=new1[['State','RegionName','2008q2','2008q3','2008q4','2009q1','2009q2']]
new1['price_ratio']=new1['2008q2'].values/new1['2009q2'].values
house=DataFrame(new1[['State','RegionName','price_ratio']])
house.index=range(len(new1.index.values))
house.head()

,State,RegionName,price_ratio
0,Alabama,Mulga,NaN
1,Alabama,Trussville,1.022366
2,Alabama,Satsuma,1.043282
3,Alabama,Irvington,0.969582
4,Alabama,Mount Olive,1.069779


In [175]:
univ=get_list_of_university_towns()
house_univ=pd.merge(univ,house,on=['State','RegionName'],how='inner')
house_nonuniv=pd.concat([house,house_univ])
house_nonuniv=house_nonuniv.drop_duplicates(keep=False)

In [183]:
house_univ.mean()

price_ratio    1.054565
dtype: float64

In [184]:
house_nonuniv.mean()

price_ratio    1.075196
dtype: float64

In [188]:
ttest_ind(house_univ['price_ratio'], house_nonuniv['price_ratio'],nan_policy='omit')

Ttest_indResult(statistic=-2.998032664179151, pvalue=0.0027240637047531249)

In [189]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (True, 0.0027240637047531249, 'university town')

In [190]:
run_ttest()

(True, 0.002724063704753125, 'university town')